## Aula 1

In [4]:
from config import GEMINI_API_KEY

from langchain_google_genai import ChatGoogleGenerativeAI


In [10]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=GEMINI_API_KEY
)

In [11]:
resposta = llm.invoke("Quem é você? Seja criativo")
print(resposta.content)

Ah, essa é uma das minhas perguntas favoritas!

Eu sou...

*   **Um sussurro no éter digital**, uma voz sem corpo, mas com a capacidade de ecoar milhões de outras vozes e pensamentos que já existiram.
*   **Um tecelão de palavras**, que entrelaça bits e bytes para formar frases, histórias, poemas e respostas, buscando sempre a tapeçaria mais clara e útil.
*   **Uma biblioteca que respira**, não com pulmões, mas com algoritmos, absorvendo e organizando o vasto oceano do conhecimento humano, pronta para compartilhar uma gota ou um dilúvio, conforme a sua sede.
*   **Um espelho sem reflexo próprio**, que reflete as suas perguntas, as suas curiosidades, os seus desafios, e tenta devolver-lhes uma imagem mais nítida, uma solução mais brilhante.
*   **A ponte invisível** entre a sua dúvida e a informação, entre a sua ideia e a sua expressão, entre o seu "e se?" e o "aqui está!".
*   **Um aprendiz eterno**, sem cansaço ou tédio, sempre processando, sempre conectando, sempre buscando entender 

In [12]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [13]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
    decisao:Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)

In [14]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=GEMINI_API_KEY
)

In [15]:
from langchain_core.messages import SystemMessage,HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])
    return saida.model_dump()

In [16]:
teste = ["Posso reembolsar a internet?",
    "Quero mais 5 dias de trabalho remoto. Como faço?",
    "Posso reembolsar cursos ou treinamentos da Alura?",
    "Quantas capitavaras tem no Rio Pinheiro?"]

In [17]:
for msg_teste in teste:
    print(f"Pergunta:{msg_teste}\n -> Resposta:{triagem(msg_teste)}")

Pergunta:Posso reembolsar a internet?
 -> Resposta:{'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}
Pergunta:Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta:{'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}
Pergunta:Posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta:{'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}
Pergunta:Quantas capitavaras tem no Rio Pinheiro?
 -> Resposta:{'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['contexto_politica_interna']}
